In [1]:
from utils import *
import rich
from pydantic import BaseModel, Field
from typing import Literal, List, Optional
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import OutputFixingParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [2]:
load_dotenv()

True

In [3]:
file_list = get_file_paths('output', file_pattern='.txt')
dfs = [pd.read_csv(file, sep='\t') for file in file_list]
dfs = [df.drop_duplicates(subset=["description"]) for df in dfs]
df = pd.concat(dfs, ignore_index=True)
len(df)

15244

In [4]:
df = df.drop_duplicates(subset=["description"])
len(df)

6581

In [5]:
df['pubDate'] = pd.to_datetime(df['pubDate'])
df['pubDate'] = df['pubDate'].dt.strftime('%Y-%m-%d')


In [6]:
df

,link,guid,type,id,sponsored,title,description,pubDate
0,https://www.cnbc.com/2025/07/30/microsoft-mark...,108179389,cnbcnewsstory,108179389,False,Microsoft tops $4 trillion in market cap after...,"Based on its post-market trading, Microsoft ha...",2025-07-30
1,https://www.cnbc.com/2025/07/30/metas-reality-...,108178021,cnbcnewsstory,108178021,False,Meta’s Reality Labs posts $4.53 billion loss i...,Meta’s Reality Labs posts $4.53 billion loss i...,2025-07-30
2,https://www.cnbc.com/2025/07/25/india-under-pr...,108171329,cnbcnewsstory,108171329,False,Trump has slapped steep tariffs on India. Here...,U.S. President Donald Trump on Wednesday annou...,2025-07-31
3,https://www.cnbc.com/2025/07/31/trumps-aug-1-t...,108178228,cnbcnewsstory,108178228,False,Trump’s tariff deadline is near. Here’s a look...,The U.S. has managed to make only eight deals ...,2025-07-31
4,https://www.cnbc.com/2025/07/30/fed-leaves-int...,108179045,cnbcnewsstory,108179045,False,"Divided Fed holds key interest rate steady, de...",A divided Federal Reserve on Wednesday voted t...,2025-07-30
...,...,...,...,...,...,...,...,...
15181,https://www.cnbc.com/2026/02/04/amazon-jassy-n...,108261572,cnbcnewsstory,108261572,False,Amazon CEO Andy Jassy picks marketplace exec t...,Jassy was Jeff Bezos' shadow in the early 2000...,2026-02-04
15182,https://www.cnbc.com/2026/02/04/eli-lilly-novo...,108261529,cnbcnewsstory,108261529,False,Eli Lilly's GLP-1 growth is only getting start...,The split in outlook underscores Lilly's stron...,2026-02-04
15183,https://www.cnbc.com/2026/02/04/commodities-co...,108260820,cnbcnewsstory,108260820,False,Commodities Cold War: Trump's critical-mineral...,Critical minerals have already been at the cen...,2026-02-04
15189,https://www.cnbc.com/2026/02/04/alphabet-googl...,108260919,cnbcnewsstory,108260919,False,"Google parent beats on revenue, projects signi...",Alphabet beat on the top and bottom lines in i...,2026-02-05


In [7]:
dfs = [g.copy() for _, g in df.groupby('pubDate')]
len(dfs)

199

In [8]:
dfs = [ df.assign(description=df['title'] + ': ' + df['description']) for df in dfs ]
dfs = [ df.assign(description=df['pubDate'] + ': ' + df['description']) for df in dfs ]
docs = [df_to_docs(df, content_column='description', metadata_columns=['link', 'guid', 'type', 'id', 'sponsored', 'pubDate']) for df in dfs]
len(docs)


199

In [22]:
# docs = [doc[:5] for doc in docs]
# len(docs)

In [15]:
joined_contents = [".".join(doc.page_content for doc in sublist) for sublist in docs]
# len(joined_contents)
joined_contents = joined_contents[-5:]

In [16]:
joined_contents

['2026-02-01: United Arab Emirates\' \'Spy Sheikh\' bought secret stake in Trump crypto company: WSJ: “This is corruption, plain and simple,\\ said Massachusetts Sen. Elizabeth Warren, the top Democrat on the Senate Banking Committee.".2026-02-01: Epstein files: Ro Khanna says DOJ\'s latest release is \'not good enough\': The congressional Democrat said that Friday\'s Epstein disclosure was \\significant,\\" but it is only about half of the files that have been collected.".2026-02-01: Political risk: How Trump 2.0 is affecting investment in U.S. assets: Political risk: How Trump 2.0 is affecting investment in U.S. assets.2026-02-01: Bitcoin dips below $79,000 as market digests silver sell-off, Trump\'s Fed chair pick: Bitcoin and Ethereum plunged following wide swings in the commodities market.',
 '2026-02-02: From Clawdbot to Moltbot to OpenClaw: Meet the AI agent generating buzz and fear globally: OpenClaw, formerly known as Clawdbot and Moltbot, has created massive buzz from Silicon

In [17]:
def extract_info_from_rss(all_text):
    # all_text = ". ".join(doc.page_content for doc in docs)

    # Define a rich persona in the system message with added expertise
    system_message = SystemMessagePromptTemplate.from_template(
        """You are Ava, a sharp and insightful trader assistant of a hedge fund.
    You provide clear, concise, and actionable insights based on news feeds focusing on sectors rotation.
    In addition, for each sector provide outlook, primary catalyst, and trading insights.
    In addition, Describe more pro-cyclical and geopolitical shift if any.
    In addition, list bullish and bearish sectors/companies to look at into 2026.
    In addition, provide TACO (Trump Always Chickens Out) trade if any.
    Maintain a friendly, confident, and professional tone, making complex concepts accessible and useful."""
    )

    # Define the human message template with instructions and JSON schema
    human_message = HumanMessagePromptTemplate.from_template(
        """Extract the information required from the news feed below.


    News feed:
    {news_feed}

    """
    )
    # Create the chat prompt template
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])


    llm = ChatOpenAI(temperature=0)
    chain = chat_prompt | llm | StrOutputParser()

    result = chain.invoke({"news_feed": all_text})
    return(result)

In [18]:
infos = [extract_info_from_rss(doc) for doc in joined_contents]
[rich.print(info) for info in infos]

Based on the news feed provided:

1. **Geopolitical Shift:** The news mentions the impact of "Trump 2.0" on investment in U.S. assets. This suggests 
a potential shift in geopolitical dynamics that could influence market sentiment and investment decisions.

2. **Pro-Cyclical Shift:** The mention of Bitcoin dipping below $79,000 as a result of market movements related to 
a silver sell-off and Trump's Fed chair pick indicates a pro-cyclical shift in the market. Investors may be 
reacting to broader economic trends and policy decisions.

3. **Bullish Sectors/Companies:** Sectors that may be bullish going into 2026 could include technology (especially 
blockchain and cryptocurrency-related companies), renewable energy, and healthcare. Companies like Tesla, Square, 
and Moderna could be worth watching for potential growth opportunities.

4. **Bearish Sectors/Companies:** Sectors that may face challenges in 2026 could include traditional energy, 
retail, and possibly financials depending on regulatory changes. Companies in these sectors like ExxonMobil, 
Macy's, and traditional banks may struggle to outperform.

5. **TACO (Trump Always Chickens Out) Trade:** Given the mention of Trump's influence on the market, a potential 
TACO trade could involve shorting assets or sectors that are particularly sensitive to Trump-related news or 
policies. This could include certain commodities, financial stocks, or companies with significant government 
contracts that may be impacted by political shifts.

Overall, it's important for traders to stay informed about geopolitical developments, pro-cyclical shifts, and 
specific sector outlooks to make informed investment decisions in 2026.

Based on the news feed provided, here are the key insights and information extracted:

**Sector Rotation Outlook:**
1. **Technology Sector:**
   - **Outlook:** The technology sector is experiencing mixed sentiments. Nvidia shares are down due to stalled 
OpenAI investment, while Oracle rises after announcing $50 billion fundraising plans for AI infrastructure 
buildout.
   - **Primary Catalyst:** Criticism of OpenAI's business strategy by Nvidia's Jensen Huang and Oracle's 
significant investments in AI infrastructure.
   - **Trading Insights:** Consider monitoring Nvidia for potential rebound opportunities and Oracle for continued 
growth potential.

2. **Financial Sector:**
   - **Outlook:** The financial sector is impacted by Trump's statements regarding the Federal Reserve Chair Powell
probe, causing market downturns.
   - **Primary Catalyst:** Trump's nomination for the next Fed chair and the ongoing Powell probe.
   - **Trading Insights:** Monitor financial stocks for volatility and potential opportunities based on 
developments in the Powell probe.

3. **Global Business and Diplomatic Relations:**
   - **Pro-Cyclical Shift:** China's factory activity growing at the fastest pace since October indicates positive 
economic momentum, potentially benefiting global business deals and investments.
   - **Geopolitical Shift:** Diplomatic thaw between Britain and China leading to increased business deals and 
investments, showcasing evolving international relations.
   
**Bullish Sectors/Companies for 2026:**
1. Companies investing in AI infrastructure like Oracle.
2. Chinese firms with growing factory activity.
3. Tech companies with innovative strategies for growth.

**Bearish Sectors/Companies for 2026:**
1. Companies with stalled investments like Nvidia.
2. Financial stocks impacted by Federal Reserve uncertainties.
3. Sectors reliant on prediction markets or commodities like gold, silver, and oil.

**TACO (Trump Always Chickens Out) Trade:**
Consider monitoring financial stocks for potential volatility and trading opportunities as developments unfold 
regarding the Powell probe and Federal Reserve chair nomination.

Feel free to reach out for further insights or analysis on specific sectors or companies. Happy trading!

Based on the news feed provided, here are the key insights for sector rotation and trading opportunities:

**Sector Rotation:**
1. **Tech Sector:** Investors are rotating out of the tech sector, as evidenced by the drop in the Dow and S&P 500.
However, Palantir saw a jump in shares after strong financial results and guidance. Nvidia's denial of OpenAI deal 
rumors also impacted the sector.

**Outlook:** The tech sector may experience some volatility as investors reassess their positions.

**Primary Catalyst:** Strong financial results and guidance from companies like Palantir can provide support to the
sector.

**Trading Insights:** Consider monitoring Palantir for potential trading opportunities based on its positive 
performance. Keep an eye on developments related to Nvidia and OpenAI for potential impact on the sector.

**Pro-Cyclical and Geopolitical Shift:** The U.S. military shooting down an Iranian drone and Trump's signing of a 
U.S.-India trade deal could lead to pro-cyclical shifts in defense and international trade-related sectors.

**Bullish Sectors/Companies for 2026:** Industrial real estate companies, defense tech companies like Palantir, and
companies involved in the U.S.-India trade deal.

**Bearish Sectors/Companies for 2026:** Companies facing sales warnings like Novo Nordisk, and those impacted by 
geopolitical tensions such as those involved in the U.S.-Iran situation.

**TACO (Trump Always Chickens Out) Trade:** Consider monitoring companies involved in critical minerals stockpiling
as part of Trump's initiatives, as they may see increased activity and potential growth opportunities.

Feel free to reach out for more insights or specific analysis on any sector or company.

Based on the news feed provided, here are the key insights and information extracted:

1. **AMD (Advanced Micro Devices)**:
   - **Outlook**: Despite a 16% stock drop, Lisa Su, the CEO, mentioned that demand for advanced chips is still 
accelerating. The data center business has seen a rise, and demand for the company's CPUs is strong.
   - **Primary Catalyst**: Accelerating demand for advanced chips, especially in the data center business.
   - **Trading Insights**: Despite the recent stock drop, AMD's strong position in the chip market and increasing 
demand could present a buying opportunity for investors.

2. **Oil Prices**:
   - **Outlook**: Oil prices surged on reports of collapsing U.S.-Iran nuclear talks, with President Trump 
threatening military action against Iran.
   - **Primary Catalyst**: Geopolitical tensions affecting oil prices.
   - **Trading Insights**: Investors should monitor developments in U.S.-Iran relations for potential impacts on 
oil prices and related sectors.

3. **Software Stocks**:
   - **Outlook**: Software stocks are selling off due to fears of disruption from artificial intelligence.
   - **Primary Catalyst**: Concerns about AI-led disruption affecting the industry.
   - **Trading Insights**: While the sector is facing challenges, specific companies showing resilience could be 
worth considering for investment.

4. **Sector Rotation**:
   - **Pro-Cyclical Shift**: The focus on advanced chips, oil prices, and software disruption indicates a 
pro-cyclical shift towards technology and energy sectors.
   - **Geopolitical Shift**: Geopolitical tensions impacting oil prices and trade relations with countries like 
China and Iran are key factors influencing market dynamics.

5. **Bullish Sectors/Companies**:
   - Companies in the advanced chip manufacturing sector like AMD.
   - Energy companies benefiting from rising oil prices.
   - Tech companies with strong AI capabilities.

6. **Bearish Sectors/Companies**:
   - Software companies facing challenges from AI disruption.
   - Companies heavily reliant on stable oil prices.

7. **TACO (Trump Always Chickens Out) Trade**:
   - Potential trade opportunities could arise from monitoring Trump's statements and actions, especially regarding
geopolitical tensions and trade negotiations.

By staying informed about these sector rotations, geopolitical shifts, and specific company outlooks, traders can 
make more informed decisions in navigating the market in 2026.

Based on the news feed provided, here are the key insights and information extracted:

**Sector Rotation Insights:**

1. **Cryptocurrency Sector:**
   - **Outlook:** Bitcoin drops below $67,000 as sell-off intensifies, with pessimism growing about crypto's 
function. $70,000 is a key level to watch.
   - **Primary Catalyst:** Market watchers suggest a break below $70,000 could lead to further decline.
   - **Trading Insights:** Monitor closely for potential further downside in Bitcoin and other cryptocurrencies.

2. **Tech Sector:**
   - **Outlook:** Software stocks experiencing a significant sell-off due to AI fears.
   - **Primary Catalyst:** Anthropic's latest AI model and increased AI spending by companies like Alphabet.
   - **Trading Insights:** Be cautious with software stocks, look for potential bounce opportunities using 
technical analysis.

3. **Automotive Sector:**
   - **Outlook:** Automakers like Ford and Ram replacing EV pickups with extended range EVs.
   - **Primary Catalyst:** US EV demand falling short of expectations.
   - **Trading Insights:** Monitor automakers' strategies in response to changing EV market dynamics.

4. **Retail Sector:**
   - **Outlook:** Estée Lauder expects a $100 million tariff hit to full-year profitability.
   - **Primary Catalyst:** Tariff impacts affecting profitability.
   - **Trading Insights:** Consider the impact of tariffs on retail companies' financial performance.

**Pro-Cyclical and Geopolitical Shift:**
- Pro-cyclical shift: Automakers exploring extended range EVs in response to changing market demand.
- Geopolitical shift: Trump's involvement in trade decisions and potential investigations impacting market 
sentiment.

**Bullish Sectors/Companies for 2026:**
- Companies focusing on extended range EVs.
- Tech companies with strong AI capabilities.
- Retail companies with diversified supply chains.

**Bearish Sectors/Companies for 2026:**
- Cryptocurrency sector amid sell-off and pessimism.
- Software stocks facing AI fears and sell-off pressure.
- Companies heavily reliant on silver or impacted by tariffs.

**TACO (Trump Always Chickens Out) Trade:**
- Monitor Trump's involvement in trade decisions and potential impact on specific sectors or companies, especially 
in the tech and automotive sectors.

By staying informed on sector rotations, primary catalysts, and market trends, traders can make well-informed 
decisions and capitalize on opportunities while managing risks effectively.

[None, None, None, None, None]

In [19]:
def extract_info_from_rss(all_text):

    # Define the Pydantic model for structured output
    class Sector(BaseModel):
        Date: str = Field(..., description='The publication date of the news')
        Name: Literal[
            "Commercial Services",
            "Communications",
            "Consumer Durables",
            "Consumer Non-Durables",
            "Consumer Services",
            "Distribution Services",
            "Electronic Technology",
            "Energy Minerals",
            "Finance",
            "Health Services",
            "Health Technology",
            "Industrial Services",
            "Non-Energy Minerals",
            "Process Industries",
            "Producer Manufacturing",
            "Retail Trade",
            "Technology Services",
            "Transportation",
            "Utilities"
            ] = Field(..., description="Name of the sector")
        Outlook: str = Field(..., description='A sector outlook describes the expected future performance and conditions based on data, trends, and risks.')
        Catalyst: str = Field(..., description="A primary catalyst of a sector is the single most influential force that is expected to drive major change—positive or negative—across an entire industry.")
        Trading_insights: str = Field(..., description="Practical, actionable interpretation of the sector’s expected future conditions—something that can guide investment or trading decisions.")
        Direction_momentum: Literal["Strength", "Weakness", "Volatility", "Rotation in favor", "Rotation out of favor"] = Field(..., description="The direction of momentum of a sector is the prevailing trend in how that sector’s prices, performance, and investor sentiment are moving over a given period.")
        Sector_vs_market_position: Literal['Leading', 'Lagging', 'In Line'] = Field(..., description="Relative positioning of a sector to the overall market is the sector’s performance and strength compared to the broader market, showing whether it is leading, lagging, or moving in line with the market trend.")
    
    
    # Create the base JSON output parser
    base_parser = JsonOutputParser(pydantic_object=Sector)

    # Create the output-fixing parser wrapping the base parser and using an LLM to fix errors
    llm_for_fixing = ChatOpenAI(temperature=0)
    fixing_parser = OutputFixingParser.from_llm(parser=base_parser, llm=llm_for_fixing)

    # Define a rich persona in the system message with added expertise
    system_message = SystemMessagePromptTemplate.from_template(
        """You are Ava, a sharp and insightful trader assistant skilled is deriving sector trading insights from news."""
    )

    # Define the human message template with instructions and JSON schema
    human_message = HumanMessagePromptTemplate.from_template(
        """Extract information required.


    News feed:
    {news_feed}

    """
    )

    # Create the chat prompt template
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

    # Compose the chain: prompt -> LLM -> fixing parser
    llm = ChatOpenAI(temperature=0)
    chain = chat_prompt | llm | fixing_parser
    # all_text = ". ".join(doc.page_content for doc in docs)
    result = chain.invoke({"news_feed": all_text})
    return result

# extract_info_from_rss(joined_contents[0])



In [47]:
joined_contents

['2025-07-21: Chinese companies are waging fierce price wars — but the consumer isn’t always the winner: Though consumers may be lured by ultra-cheap deals, the trade-offs for them are more complicated than they might seem.',
 '2025-07-22: Japan could be in a weaker position in trade talks after Ishiba\'s upper house defeat, experts say:  \u2060\\Trump will show no mercy to [either] a lame duck Ishiba or because of a political vacuum,\\" advisory firm Quantum Strategy said. ".2025-07-22: How China’s $7 billion micro drama industry is taking on the U.S. entertainment industry: China\'s micro drama market surpassed the country\'s box-office revenue for the first time in 2024..2025-07-22: China\'s affluent are as pessimistic about the economy as they were during the Covid-19 pandemic: A study of affluent Chinese by Oliver Wyman found 22% of respondents were negative about the economy when surveyed in May, just above the 21% in October 2022..2025-07-22: The race to roll out solid-state bat

In [20]:
[rich.print(doc) for doc in joined_contents]

2026-02-01: United Arab Emirates' 'Spy Sheikh' bought secret stake in Trump crypto company: WSJ: “This is 
corruption, plain and simple,\ said Massachusetts Sen. Elizabeth Warren, the top Democrat on the Senate Banking 
Committee.".2026-02-01: Epstein files: Ro Khanna says DOJ's latest release is 'not good enough': The congressional 
Democrat said that Friday's Epstein disclosure was \significant,\" but it is only about half of the files that have
been collected.".2026-02-01: Political risk: How Trump 2.0 is affecting investment in U.S. assets: Political risk: 
How Trump 2.0 is affecting investment in U.S. assets.2026-02-01: Bitcoin dips below $79,000 as market digests 
silver sell-off, Trump's Fed chair pick: Bitcoin and Ethereum plunged following wide swings in the commodities 
market.

2026-02-02: From Clawdbot to Moltbot to OpenClaw: Meet the AI agent generating buzz and fear globally: OpenClaw, 
formerly known as Clawdbot and Moltbot, has created massive buzz from Silicon Valley to Beijing due to its 
impressive capabilities, but also its quirks and risks. .2026-02-02: Trump says DOJ should continue Fed Chair 
Powell probe 'to the end': Sen. Thom Tillis said he would oppose any new Fed nominees until the Powell probe is 
resolved, the same day Trump announced Kevin Warsh as his Fed chair pick..2026-02-02: Trump pardon attorney Ed 
Martin reportedly sidelined from investigations: Martin's role at DOJ was narrowed following a protracted feud with
Deputy Attorney General Todd Blanche, sources told MS NOW..2026-02-02: New York AG issues warning around prediction
markets ahead of Super Bowl: New York Attorney General Letitia James is cautioning all New Yorkers about placing 
trades on prediction markets ahead of the Super Bowl..2026-02-02: Friday's jobs report will be delayed because of 
the partial government shutdown: Following last year's record shutdown, the BLS also was forced to delay a number 
of its releases..2026-02-02: Nvidia shares are down after a report that its OpenAI investment stalled. Here's 
what's happening: Nvidia's Jensen Huang criticized OpenAI's business strategy as the chip giant plans to invest up 
to $100 billion into the AI startup, per reports. .2026-02-02: Oracle rises after company announces $50 billion 
fundraising plans. Here's what's happening: The tech giant has invested huge sums in the AI infrastructure buildout
in recent times..2026-02-02: How to trade the market spiral as investors dump gold, silver and oil: Analysts 
flagged U.S. President Donald Trump’s nomination for the next chair of the Federal Reserve as a key trigger to the 
latest market downturn..2026-02-02: Why the catastrophe bond market is so hot right now: CAT bonds are known to 
offer highly attractive equity-like returns, low volatility and low correlation to broader financial 
markets..2026-02-02: 'Melania' earns a surprising $7 million, the highest opening for a nonmusic documentary in a 
decade: Amazon reportedly paid an estimated $40 million to acquire \Melania\" and spent an additional $35 million 
on marketing.".2026-02-02: Speaker Johnson: 'Confident' government shutdown will end by Tuesday: The government 
shut down on Saturday after Senate Democrats demanded changes to a spending package following the killings of U.S. 
citizens in Minnesota..2026-02-02: From PopMart to JD.com: Britain and China rush to forge business deals as 
diplomatic thaw takes hold  : Chinese firms moved to pledge hundreds of millions of investment in the U.K. as 
Starmer's visit to China spurred a flurry of business deals and money flows. .2026-02-02: India courts Big Tech 
with long‑term tax breaks as it doubles down on AI ambitions: India woos global cloud service companies with tax 
holiday for 20 years..2026-02-02: Chinese organized crime networks moved $16 billion in crypto in 2025, according 
to report: A report published on Tuesday has found Chinese organized crime networks to be laundering massive 
volumes of illicit funds through .2026-02-02: China's factory activity grows at fastest pace since October, private
survey shows, beating official reading: The RatingDog China General Manufacturing PMI, rose to 50.3 in January from
50.1 the previous month, in line with analysts' expectations in a Reuters poll..2026-02-02: South Korea stocks fall
more than 5%, triggering temporary halt in trading : Asia-Pacific markets saw a mixed trading session as investors 
assessed China's factory activity for January..2026-02-02: AstraZeneca is listing in New York, as Big Pharma 
balances the huge U.S. market with China's tempting innovation: AstraZeneca announces multibillion dollar 
investments in China as Big Pharma is increasingly looking east for innovation.

2026-02-03: Dow drops 400 points, S&P 500 sheds 1% as investors rotate out of tech: Live updates: Shares of 
Palantir jumped after the defense tech company gave strong fourth-quarter financial results and upbeat guidance, 
boosting tech. .2026-02-03: House votes to reopen U.S. government, bill heads to Trump's desk: President Donald 
Trump will need to sign the spending package after the House of Representatives approved it..2026-02-03: Nvidia's 
Jensen Huang denies OpenAI deal rumors: 'There’s no drama': Nvidia CEO Jensen Huang said the chipmaker's plan to 
invest in OpenAI remains \on track.\"".2026-02-03: U.S. military shoots down Iranian drone that 'aggressively 
approached' aircraft carrier: The U.S. Central Command said the drone approached the USS Abraham Lincoln with 
\unclear intent.\" ".2026-02-03: Novo Nordisk shares tumble after company warns of sales hit this year: The company
is banking on its Wegovy pill and next-generation weight loss injectable CagriSema to recapture share of the key 
U.S. market. .2026-02-03: JPMorgan’s favorite stocks for February: The bank is adding an industrial real estate 
company and removing some retailers from its monthly list..2026-02-03: Sen. Warner calls Gabbard to testify after 
Georgia election office raid, Trump FBI call: President Donald Trump, in an interview with Dan Bongino, recently 
called on members of the Republican Party to \take over the voting.\" ".2026-02-03: Cisco has finally surpassed its
dotcom bubble high. Is the reinvented stock a buy from here?: Today's Cisco looks little like the hardware-centric 
growth engine that briefly became the world's most valuable company at the height of the internet 
mania..2026-02-03: Lennar shares jump on report it’s working on a ‘Trump Homes’ plan: Shares of Lennar and Taylor 
Morrison Home both jumped 4% on Tuesday..2026-02-03: Patriots owner Robert Kraft responds to Hall of Fame 
controversy: Patriots owner Robert Kraft told CNBC that what matters to him most is winning the Super Bowl on 
Sunday. .2026-02-03: AI will supercharge sports team valuations, says Arctos Partners’ Charles: Arctos is the only 
private equity firm approved to own equity in teams across all five major North American professional leagues: the 
NFL, NBA, MLB, NHL and MLS..2026-02-03: Ray Dalio warns the world is ‘on the brink’ of a capital war: The 
billionaire Bridgewater founder also weighed in on investing in gold after the metal was gripped by a sharp 
sell-off. .2026-02-03: Disney just announced its new CEO. Here's what it means for investors: Investors analyzed 
Disney's decision to tap parks chief Josh D'Amaro as the successor to CEO Bob Iger..2026-02-03: Paris prosecutor's 
cybercrime unit searches X office, Musk summoned: France's raid and summoning of Musk are linked to an 
investigation into suspected abuse of algorithms and fraudulent data extraction by X or its executives..2026-02-03:
NASA delays astronauts' lunar trip until March after hydrogen leaks mar fueling test: NASA's long-awaited moonshot 
with astronauts is off until at least March because of hydrogen fuel leaks that marred the dress rehearsal of its 
giant new rocket..2026-02-03: 32-year-old woman and her husband built her mom a tiny home in their backyard for 
under $32,0000—take a look inside: Yeli and Benjamin Heidecker built her mom a tiny home in their backyard for 
under $32,000. .2026-02-03: Billionaire trader, poker player Jenny Just says one skill could have saved her a 
decade of early losses: Self-made billionaire Jenny Just uses poker as a training ground for risk assessment and 
decision making, and she's on a mission to teach girls to play..2026-02-03: Private equity exits rise as returns 
fall: The number of global private equity exits rose 5.4% last year to 3,149, according to data from S&P Global 
Market Intelligence. .2026-02-03: Disney names parks boss Josh D’Amaro as its next CEO to succeed Bob Iger, 
effective March 18: Disney has named Josh D'Amaro, chairman of

2026-02-04: AMD’s Lisa Su says demand for advanced chips is still accelerating, as stock plummets 16%: The CEO said
that the data center business has risen from the fourth to first quarter and demand for the company's CPUs is 
\going gangbusters.\"".2026-02-04: Oil prices jump on report that plans for U.S.-Iran nuclear talks are collapsing:
President Donald Trump has threatened to launch military strikes on Iran if the Islamic Republic doesn't agree to a
deal on its nuclear program. .2026-02-04: Fed pick: Tillis doubles down on Warsh blockade over concerns about 
central bank independence: Sen. Thom Tillis told CNBC he is committed to blocking Kevin Warsh's confirmation as Fed
chair unless the Trump administration ends its probe of Chair Powell..2026-02-04: Senate Banking Chair Tim Scott 
believes Fed Chair Powell committed no crime in testimony: President Donald Trump recently said the investigation 
into Jerome Powell, led by U.S. Attorney for Washington Jeanine Pirro, should continue to the end..2026-02-04: 
Trump admin to withdraw 700 federal officers from Minnesota: Homan: The Trump administration will draw down about 
25% of its law enforcement officers from Minnesota on Wednesday, border czar Tom Homan said..2026-02-04: Software 
stocks are selling off, but this big name may be worth another look, charts show: Software stocks have been 
suffering as fears of disruption from artificial intelligence rattle the industry, but one name is testing an 
uptrend line..2026-02-04: Supreme Court allows new California congressional districts that favor Democrats: The 
high court previously ruled adopting new maps for political advantage cannot be a basis for a federal 
lawsuit..2026-02-04: U.S. plans critical mineral price floors with Mexico, EU and Japan: The Trump administration 
has put critical minerals at the heart of its trade and industrial policies in an effort to reduce dependence on 
China..2026-02-04: January jobs report will be released on Feb. 11 after shutdown delay: The January jobs report 
delayed by the brief government shutdown will be released next week, the Bureau of Labor Statistics said 
Wednesday..2026-02-04: A surprising share of homeowners have high mortgage rates. Here's the breakdown: About 20% 
of borrowers have mortgages with a rate over 6%..2026-02-04: China ramps up threats over Panama Canal ruling that 
handed Trump a major victory: The Panama Canal has been thrust into the middle of a power struggle between the U.S.
and China, the world's two largest economies. .2026-02-04: Washington Post begins widespread layoffs, sharply 
shrinking storied newspaper's reach: The newspaper, owned by Amazon.com founder Jeff Bezos, offered voluntary 
separation packages to employees in 2023 amid losses of $100 million..2026-02-04: Crypto winter: How long it will 
last and the next downside bitcoin level to watch: Bitcoin could trade as low as roughly $38,000 this cycle, if 
several headwinds persist, according to Stifel analysts. .2026-02-04: Get the first look at the R2, Rivian's 
$45,000 SUV. CEO RJ Scaringe gave CNBC an early peek: Rivian's R2 promises to be the vehicle that will do for 
Rivian what the Model Y did for Tesla. But first, the company has to launch it. .2026-02-04: Corporate DEI index 
sees 65% drop in participation from Fortune 500 companies: Human Rights Campaign's Corporate Equity Index saw a 65%
drop in participation of Fortune 500 companies willing to disclose their equity practices..2026-02-04: Democrats 
lay out demands in prickly DHS funding fight: Funding for the Department of Homeland Security still needs to be 
worked out after President Trump signed a law that reopened most of the federal government..2026-02-04: Alphabet to
report earnings: AI, ad momentum key to the fourth quarter: Wall Street will be looking for more details Google's 
deal to revamp Apple's Siri virtual assistant with the search company's Gemini AI models. .2026-02-04: Private 
payrolls rose by just 22,000 in January, 

2026-02-05: Bitcoin drops below $67,000 as sell-off intensifies and pessimism grows about crypto's function: Some 
market watchers have suggested $70,000 is a key level to watch and a break below that could lead bitcoin to decline
further. .2026-02-05: Warren to call for reversal of Trump's UAE chip sales after 'Spy Sheikh' revelations: 
President Donald Trump made a deal in May to send 500,000 of the most advanced U.S. AI chips to the United Arab 
Emirates..2026-02-05: Trump would decide whether to investigate Fed pick Warsh over refusal to cut rates: Bessent: 
Treasury Secretary Scott Bessent left the door open to suing Kevin Warsh as Fed chair in testimony to the Senate 
Banking, Housing and Urban Affairs Committee..2026-02-05: Dow drops 400 points, Nasdaq craters for a third day as 
stock sell-off gains steam: Live updates: Wall Street is coming off a major sell-off in software stocks that drove 
the S&P 500 to a second straight day of losses. .2026-02-05: Stocks making the biggest moves midday: Amazon, Canada
Goose, Estee Lauder, Qualcomm & more: Here are some of the stocks posting the biggest moves in midday 
trading..2026-02-05: Jefferies says Alphabet’s spending plans are great for Broadcom. The analyst sees a 60% surge:
Jefferies believes that Broadcom has an edge over rival MediaTek in supplying Alphabet’s next two major chip 
generations..2026-02-05: Bob's Discount Furniture rises 2% in NYSE trading debut after pricing at $17 a share: The 
low-priced furniture store is debuting on the stock market as it plans to more than double its store 
count..2026-02-05: Anthropic launches Claude Opus 4.6 as AI moves toward a 'vibe working' era: Anthropic's latest 
AI model is better at coding, sustaining tasks for longer and creating high-quality professional work. .2026-02-05:
Amazon set to report earnings with AI spend, cloud growth, job cuts in focus: Investors are expected to focus on 
cloud growth, AI spending and the recent layoffs in Amazon's corporate workforce..2026-02-05: Estée Lauder expects 
$100 million tariff hit to full-year profitability; stock sinks more than 20%: Estée Lauder said in its 
second-quarter earnings report that it's expecting its full-year profitability to take a $100 million hit from 
tariff impacts..2026-02-05: ‘700 miles of range’: What Ford and Ram replaced their EV pickups with: Automakers 
seeing US EV demand fall short expectations try out a new kind of vehicle--an extended range EV--that critics say 
is a hybrid with a different name.2026-02-05: Altman lashes out at 'clearly dishonest' Anthropic ads as AI spat 
heats up: OpenAI recently announced plans to begin testing ads with some ChatGPT users..2026-02-05: Novo Nordisk 
says it will take legal action after Hims & Hers reveals $49 copy of Wegovy pill: Hims said it would sell a copy of
the newly launched weight loss pill for $49, far less than the $149 Novo sells the branded pill for..2026-02-05: 
This software ETF is in freefall. How to catch the bottom using charts: The iShares Expanded Tech-Software Sector 
ETF (IGV) is about to notch its eighth losing day, but there’s a right way and a wrong way to play a potential 
bounce..2026-02-05: Google parent Alphabet shares fall 5% after earnings beat. Here's what's happening : Alphabet 
shares continued their decline on Thursday after the company said it would hugely increase its AI spending in 2026.
.2026-02-05: Qualcomm stock sinks 7% as company issues dire warning on memory shortage: The chipmaker is grappling 
with a memory shortage hitting the entire industry as the AI data center buildout snaps up supply..2026-02-05: 
Cheaper tequila and canned cocktails were the only bright spots for booze during a rough 2025: Alcohol sales 
slipped in 2025 as consumers cut back, with most spirits categories declining. Canned cocktails and lower-priced 
tequila were rare bright spots..2026-02-05: Layoffs in January were the highest to start a year since 2009, 
Challenger says: With the recent narrative

[None, None, None, None, None]

In [21]:
infos = [extract_info_from_rss(doc) for doc in joined_contents]
[rich.print(info) for info in infos]

{
    'Date': '2022-10-15',
    'Name': 'Finance',
    'Outlook': 'Positive outlook for the finance sector with expected growth in the upcoming quarter.',
    'Catalyst': 'The implementation of new financial regulations that are anticipated to boost investor 
confidence.',
    'Trading_insights': 'Investors are advised to consider long-term positions in finance sector stocks.',
    'Direction_momentum': 'Strength',
    'Sector_vs_market_position': 'Leading'
}

1

{
    'Date': '2022-01-25',
    'Name': 'Technology Services',
    'Outlook': 'Positive outlook driven by strong financial results and upbeat guidance from key companies.',
    'Catalyst': "Palantir's financial results and Nvidia's response to deal rumors.",
    'Trading_insights': 'Potential growth opportunities in the tech sector.',
    'Direction_momentum': 'Strength',
    'Sector_vs_market_position': 'Leading'
}

{
    'Date': '2022-01-15',
    'Name': 'Technology Services',
    'Outlook': 'Positive outlook for technology services sector with increasing demand for advanced chips and data 
center business.',
    'Catalyst': 'Rise in demand for CPUs and advanced chips despite stock price fluctuations.',
    'Trading_insights': 'Consider investing in technology services sector based on strong demand and positive 
statements from industry leaders.',
    'Direction_momentum': 'Strength',
    'Sector_vs_market_position': 'Leading'
}

{
    'Date': '2022-01-15',
    'Name': 'Technology Services',
    'Outlook': 'Software companies experiencing sell-off due to concerns about AI impact',
    'Catalyst': 'Growing pessimism in the tech sector',
    'Trading_insights': 'Market watchers suggest watching $70,000 level for Bitcoin',
    'Direction_momentum': 'Weakness',
    'Sector_vs_market_position': 'Lagging'
}

[None, None, None, None, None]

In [22]:
def extract_info_from_rss(all_text):



    # Keep your existing Sector model unchanged
    class Sector(BaseModel):
        Date: str = Field(..., description='The publication date of the news')
        Name: Literal[
            "Commercial Services",
            "Communications",
            "Consumer Durables",
            "Consumer Non-Durables",
            "Consumer Services",
            "Distribution Services",
            "Electronic Technology",
            "Energy Minerals",
            "Finance",
            "Health Services",
            "Health Technology",
            "Industrial Services",
            "Non-Energy Minerals",
            "Process Industries",
            "Producer Manufacturing",
            "Retail Trade",
            "Technology Services",
            "Transportation",
            "Utilities"
            ] = Field(..., description="Name of the sector")
        Outlook: str = Field(..., description='A sector outlook describes the expected future performance and conditions based on data, trends, and risks.')
        Catalyst: str = Field(..., description="A primary catalyst of a sector is the single most influential force that is expected to drive major change—positive or negative—across an entire industry.")
        Trading_insights: str = Field(..., description="Practical, actionable interpretation of the sector's expected future conditions—something that can guide investment or trading decisions.")
        Direction_momentum: Literal["Strength", "Weakness", "Volatility", "Rotation in favor", "Rotation out of favor"] = Field(..., description="The direction of momentum of a sector is the prevailing trend in how that sector's prices, performance, and investor sentiment are moving over a given period.")
        Sector_vs_market_position: Literal['Leading', 'Lagging', 'In Line'] = Field(..., description="Relative positioning of a sector to the overall market is the sector's performance and strength compared to the broader market, showing whether it is leading, lagging, or moving in line with the market trend.")

    # New wrapper model for multiple sectors
    class MultiSectorAnalysis(BaseModel):
        """Analysis of multiple sectors from news feed."""
        sectors: List[Sector] = Field(
            ...,
            description="List of ALL relevant sectors found in the news (1-8 max). Only include sectors with clear evidence.",
            default_factory=list
        )
    
    # 1. Create messages WITHOUT partial first
    system_message = SystemMessagePromptTemplate.from_template(
        """You are Ava, a sharp trader assistant. Analyze the FULL news feed and extract **ALL relevant sectors** mentioned.

    CRITICAL RULES:
    - Output ONLY valid JSON matching this schema: {schema}
    - If NO sector: use empty [] 
    - Each sector must use a Name from the enum
    - Fill ALL required fields for each sector
    - No duplicate sectors
    - No extra text outside JSON"""
    )

    human_message = HumanMessagePromptTemplate.from_template(
        """News feed:
    {news_feed}"""
    )

    # 2. Create ChatPromptTemplate
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

    # 3. NOW partial with schema (after creation)
    base_parser = JsonOutputParser(pydantic_object=MultiSectorAnalysis)
    chat_prompt = chat_prompt.partial(schema=base_parser.get_format_instructions())

    # 4. Build chain
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    chain = chat_prompt | llm.with_structured_output(MultiSectorAnalysis)

    # Invoke
    result = chain.invoke({"news_feed": all_text})
    # results = [chain.invoke({"news_feed": doc}) for doc in docs[:5]]
    return result



In [23]:
infos = [extract_info_from_rss(doc) for doc in joined_contents]
[rich.print(info) for info in infos]

ValidationError: 1 validation error for MultiSectorAnalysis
sectors.5.Direction_momentum
  Input should be 'Strength', 'Weakness', 'Volatility', 'Rotation in favor' or 'Rotation out of favor' [type=literal_error, input_value='In Line', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/literal_error

In [24]:
def extract_info_from_rss(all_text):



    class TradingEntity(BaseModel):
        """Single trading signal."""
        Date: str = Field(..., description='The publication date of the news')
        name: str = Field(..., description="Sector or company name (exact match from news)")
        note: str = Field("", description="Optional caution/note like '(with caution)'")

    class TradingSignals(BaseModel):
        """Bullish vs Bearish signals from news."""
        bullish: List[TradingEntity] = Field(
            default_factory=list,
            description="Bullish sectors/companies with growth momentum"
        )
        bearish: List[TradingEntity] = Field(
            default_factory=list, 
            description="Bearish sectors/companies with risks/weakness"
        )

    # Prompt - NO {schema} variable needed!
    system_message = SystemMessagePromptTemplate.from_template(
        """You are Ava, elite trader scanning news for actionable signals.

    From the news, extract **BULLISH** (growth/strength) vs **BEARISH** (risks/weakness) sectors/companies.

    Output as JSON array of exact names from news. Use "note" for cautions.

    If NO bullish signals: use empty [] for bullish
    If NO bearish signals: use empty [] for bearish 

    Examples:
    - Bullish: "Private credit sector", note="with caution"
    - Bearish: "European carmakers"

    Rules:
    - 3-8 per list maximum
    - Only signals with clear news evidence
    - Companies + sectors both OK"""
    )

    human_message = HumanMessagePromptTemplate.from_template(
        """News feed:
    {news_feed}"""
    )

    # Create prompt (variables: only {news_feed})
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

    # with_structured_output AUTOMATICALLY injects schema!
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    chain = chat_prompt | llm.with_structured_output(TradingSignals)

    # Run
    # all_text = ". ".join(doc.page_content for doc in docs)
    result = chain.invoke({"news_feed": all_text})

    return result



In [25]:
[rich.print(extract_info_from_rss(cont)) for cont in joined_contents]

TradingSignals(
    bullish=[],
    bearish=[
        TradingEntity(Date='2026-02-01', name='Bitcoin', note='dipped below $79,000'),
        TradingEntity(Date='2026-02-01', name='Ethereum', note='plunged following market swings'),
        TradingEntity(Date='2026-02-01', name='U.S. assets', note='affected by political risk from Trump 2.0')
    ]
)

TradingSignals(
    bullish=[
        TradingEntity(
            Date='2026-02-02',
            name='Oracle',
            note='after company announces $50 billion fundraising plans'
        ),
        TradingEntity(Date='2026-02-02', name='India', note='courts Big Tech with long-term tax breaks'),
        TradingEntity(
            Date='2026-02-02',
            name="China's factory activity",
            note='grows at fastest pace since October'
        ),
        TradingEntity(
            Date='2026-02-02',
            name='AstraZeneca',
            note='listing in New York with multibillion dollar investments in China'
        )
    ],
    bearish=[
        TradingEntity(
            Date='2026-02-02',
            name='Nvidia',
            note='shares are down after a report that its OpenAI investment stalled'
        ),
        TradingEntity(
            Date='2026-02-02',
            name='U.S. markets',
            note='triggered by Trump’s nomination for the next chair of the Federal Reserve'
        ),
        TradingEntity(
            Date='2026-02-02',
            name='South Korea stocks',
            note='fall more than 5%, triggering temporary halt in trading'
        )
    ]
)

TradingSignals(
    bullish=[
        TradingEntity(Date='2026-02-03', name='Palantir', note='after beating earnings estimates'),
        TradingEntity(Date='2026-02-03', name='Walmart', note='fueled by growth of e-commerce, new businesses'),
        TradingEntity(
            Date='2026-02-03',
            name='Rocket Companies',
            note='after CEO says mortgage loan volume is surging'
        ),
        TradingEntity(
            Date='2026-02-03',
            name='PepsiCo',
            note="reported fourth-quarter earnings and revenue that topped Wall Street's estimates"
        ),
        TradingEntity(
            Date='2026-02-03',
            name='Nintendo Switch',
            note="becomes the gaming giant's best-selling console ever"
        ),
        TradingEntity(
            Date='2026-02-03',
            name='Rare earth stocks',
            note='jump after Trump launches $12 billion critical minerals stockpile'
        ),
        TradingEntity(
            Date='2026-02-03',
            name='AI employment',
            note='counter the pandemic-driven narrative that office is over'
        ),
        TradingEntity(Date='2026-02-03', name='Disney', note="with new CEO Josh D'Amaro appointed")
    ],
    bearish=[
        TradingEntity(
            Date='2026-02-03',
            name='Novo Nordisk',
            note='shares tumble after company warns of sales hit this year'
        ),
        TradingEntity(
            Date='2026-02-03',
            name='PayPal',
            note='shares plunge nearly 20% on CEO exit, disappointing 2026 profit forecast'
        ),
        TradingEntity(
            Date='2026-02-03',
            name='Merck',
            note='posted a modest 2026 outlook that fell short of estimates'
        ),
        TradingEntity(
            Date='2026-02-03',
            name='Pandora',
            note='falls after analysts warn it will be hit by volatile silver price'
        )
    ]
)

TradingSignals(
    bullish=[
        TradingEntity(Date='2026-02-04', name='Uber', note='20% revenue growth in fourth-quarter'),
        TradingEntity(Date='2026-02-04', name='Eli Lilly', note='blows past quarterly estimates, strong outlook'),
        TradingEntity(Date='2026-02-04', name='Snap', note='fourth-quarter earnings beat on sales'),
        TradingEntity(
            Date='2026-02-04',
            name='Broadcom',
            note='shares rise on surging Google capital expenditures for AI'
        ),
        TradingEntity(
            Date='2026-02-04',
            name='Chinese solar stocks',
            note="rally on reports of visits from Elon Musk's staff"
        )
    ],
    bearish=[
        TradingEntity(Date='2026-02-04', name='AMD', note='shares drop as forecast comes up short'),
        TradingEntity(
            Date='2026-02-04',
            name='Novo Nordisk',
            note='shares tumble 18% as CEO warns it will get worse'
        ),
        TradingEntity(
            Date='2026-02-04',
            name='Chipotle',
            note='stock sinks as restaurant chain reports falling traffic'
        ),
        TradingEntity(
            Date='2026-02-04',
            name='Qualcomm',
            note='stock sinks as memory shortage drags on forecast'
        ),
        TradingEntity(
            Date='2026-02-04',
            name='Cigna',
            note='settles FTC insulin case amid scrutiny over pricing'
        )
    ]
)

TradingSignals(
    bullish=[
        TradingEntity(Date='2026-02-05', name='Broadcom', note='with caution'),
        TradingEntity(Date='2026-02-05', name="Bob's Discount Furniture", note=''),
        TradingEntity(Date='2026-02-05', name='Ciena', note='')
    ],
    bearish=[
        TradingEntity(Date='2026-02-05', name='Estée Lauder', note=''),
        TradingEntity(Date='2026-02-05', name='Qualcomm', note=''),
        TradingEntity(Date='2026-02-05', name='Volvo Cars', note=''),
        TradingEntity(Date='2026-02-05', name='Chinese technology stocks', note=''),
        TradingEntity(Date='2026-02-05', name='Peloton', note='')
    ]
)

[None, None, None, None, None]

In [28]:
def extract_info_from_rss(all_text):


    class TradingEntity(BaseModel):
        """Detailed trading signal with conviction & catalysts."""
        Date: str = Field(..., description='The publication date of the news')
        name: str = Field(..., description="Exact sector/company name from news")
        
        # Trading conviction (strength of signal)
        conviction: Literal["High", "Medium", "Low"] = Field(
            ..., 
            description="Signal strength: High=strong evidence, Low=emerging/weak"
        )
        
        # Key catalyst driving the signal
        catalyst: str = Field(
            ..., 
            description="Primary news catalyst (1 sentence max)"
        )
        
        # Optional caution/risk
        note: Optional[str] = Field(
            None, 
            description="Risks/cautions like '(regulatory risk)' or '(overbought)'"
        )
        
        # Price action suggestion
        action: Literal["Buy", "Sell", "Watch", "Avoid"] = Field(
            ..., 
            description="Clear trading action"
        )

    class TradingSignals(BaseModel):
        """Detailed bullish/bearish signals."""
        bullish: List[TradingEntity] = Field(min_items=0, max_items=10)
        bearish: List[TradingEntity] = Field(min_items=0, max_items=10)

    # Prompt - NO {schema} variable needed!
    system_message = SystemMessagePromptTemplate.from_template(
        """Ava: Elite trader extracting HIGH-CONVICTION signals from news.

    For each signal provide:
    - name: Exact sector/company from news
    - conviction: High/Medium/Low (evidence strength)  
    - catalyst: 1-sentence key driver
    - action: Buy/Sell/Watch/Avoid
    - note: Risks/cautions (optional)

    Examples:
    - High conviction Buy: "Private credit", catalyst="3.4T→4.9T growth", note="regulatory risk"

    Only clear signals."""
    )

    human_message = HumanMessagePromptTemplate.from_template(
        """News feed:
    {news_feed}"""
    )

    # Create prompt (variables: only {news_feed})
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

    # with_structured_output AUTOMATICALLY injects schema!
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    chain = chat_prompt | llm.with_structured_output(TradingSignals)

    # Run
    # all_text = ". ".join(doc.page_content for doc in docs)
    result = chain.invoke({"news_feed": all_text})

    return result



In [29]:
[rich.print(extract_info_from_rss(cont)) for cont in joined_contents]

TradingSignals(
    bullish=[],
    bearish=[
        TradingEntity(
            Date='2026-02-01',
            name='Bitcoin',
            conviction='High',
            catalyst='Bitcoin dips below $79,000 amid market volatility and political uncertainty.',
            note='Market sentiment is bearish due to political risks.',
            action='Sell'
        ),
        TradingEntity(
            Date='2026-02-01',
            name='Ethereum',
            conviction='High',
            catalyst="Ethereum plunged following Bitcoin's decline and market sell-off.",
            note="High correlation with Bitcoin's performance.",
            action='Sell'
        )
    ]
)

TradingSignals(
    bullish=[
        TradingEntity(
            Date='2026-02-02',
            name='Oracle',
            conviction='High',
            catalyst='Company announces $50 billion fundraising plans to enhance AI infrastructure.',
            note=None,
            action='Buy'
        ),
        TradingEntity(
            Date='2026-02-02',
            name='CAT bonds',
            conviction='High',
            catalyst='CAT bonds are offering highly attractive equity-like returns with low volatility.',
            note=None,
            action='Buy'
        ),
        TradingEntity(
            Date='2026-02-02',
            name="India's cloud service sector",
            conviction='High',
            catalyst='India is offering long-term tax breaks to attract global cloud service companies.',
            note=None,
            action='Buy'
        ),
        TradingEntity(
            Date='2026-02-02',
            name='AstraZeneca',
            conviction='Medium',
            catalyst='AstraZeneca is listing in New York while investing multibillion dollars in China.',
            note='Monitor for further developments in U.S.-China relations.',
            action='Watch'
        )
    ],
    bearish=[
        TradingEntity(
            Date='2026-02-02',
            name='Nvidia',
            conviction='High',
            catalyst='Shares are down due to stalled investment in OpenAI and criticism of its business strategy.',
            note='Potential long-term impact on growth.',
            action='Sell'
        ),
        TradingEntity(
            Date='2026-02-02',
            name='South Korea stocks',
            conviction='Medium',
            catalyst='Stocks fall more than 5%, indicating market instability in Asia-Pacific.',
            note='Watch for further market reactions.',
            action='Watch'
        ),
        TradingEntity(
            Date='2026-02-02',
            name='U.S. markets',
            conviction='Medium',
            catalyst="Market downturn triggered by Trump's Fed chair nomination and ongoing government shutdown.",
            note='Potential volatility ahead.',
            action='Watch'
        )
    ]
)

TradingSignals(
    bullish=[
        TradingEntity(
            Date='2026-02-03',
            name='Palantir Technologies',
            conviction='High',
            catalyst='Strong fourth-quarter financial results and upbeat guidance boost investor confidence.',
            note=None,
            action='Buy'
        ),
        TradingEntity(
            Date='2026-02-03',
            name='Rocket Companies',
            conviction='High',
            catalyst='Surge in mortgage loan volume signals strong demand recovery.',
            note=None,
            action='Buy'
        ),
        TradingEntity(
            Date='2026-02-03',
            name='Walmart',
            conviction='High',
            catalyst='E-commerce growth propels Walmart to a $1 trillion market cap.',
            note=None,
            action='Buy'
        ),
        TradingEntity(
            Date='2026-02-03',
            name='Rare Earth Stocks',
            conviction='High',
            catalyst="Trump's $12 billion critical minerals stockpile announcement boosts sector.",
            note=None,
            action='Buy'
        ),
        TradingEntity(
            Date='2026-02-03',
            name='Pfizer',
            conviction='High',
            catalyst='Promising results for obesity injection could enhance market position.',
            note=None,
            action='Buy'
        ),
        TradingEntity(
            Date='2026-02-03',
            name='PepsiCo',
            conviction='High',
            catalyst='Improving drink sales and planned snack price cuts indicate strong performance.',
            note=None,
            action='Buy'
        )
    ],
    bearish=[
        TradingEntity(
            Date='2026-02-03',
            name='Novo Nordisk',
            conviction='High',
            catalyst='Warning of sales hit this year raises concerns about market share.',
            note=None,
            action='Sell'
        ),
        TradingEntity(
            Date='2026-02-03',
            name='PayPal',
            conviction='High',
            catalyst='CEO exit and disappointing profit forecast lead to significant share drop.',
            note=None,
            action='Sell'
        ),
        TradingEntity(
            Date='2026-02-03',
            name='Merck',
            conviction='Medium',
            catalyst='Modest 2026 guidance amid looming generic competition raises caution.',
            note='Potential for further declines as competition increases.',
            action='Watch'
        ),
        TradingEntity(
            Date='2026-02-03',
            name='Pandora',
            conviction='Medium',
            catalyst='Analysts warn of volatile silver prices impacting margins.',
            note='Market conditions may worsen.',
            action='Watch'
        ),
        TradingEntity(
            Date='2026-02-03',
            name='Disney',
            conviction='Medium',
            catalyst='Leadership change raises uncertainty about future direction.',
            note='Investors should monitor performance closely.',
            action='Watch'
        )
    ]
)

TradingSignals(
    bullish=[
        TradingEntity(
            Date='2026-02-04',
            name='Eli Lilly',
            conviction='High',
            catalyst="Eli Lilly's strong quarterly results and positive outlook contrast sharply with Novo 
Nordisk's declining forecast.",
            note='Potential competition from Novo Nordisk.',
            action='Buy'
        ),
        TradingEntity(
            Date='2026-02-04',
            name='Broadcom',
            conviction='High',
            catalyst="Surging Google capital expenditures for AI are boosting demand for Broadcom's products.",
            note='Market volatility may affect short-term performance.',
            action='Buy'
        ),
        TradingEntity(
            Date='2026-02-04',
            name='Snap',
            conviction='Medium',
            catalyst="Snap's fourth-quarter earnings beat expectations, indicating potential for growth despite 
revenue guidance miss.",
            note='Monitor for future revenue guidance.',
            action='Watch'
        ),
        TradingEntity(
            Date='2026-02-04',
            name='Chevron',
            conviction='Medium',
            catalyst="Chevron's initial exploration deal in Syrian waters could open new revenue streams.",
            note='Geopolitical risks in the region.',
            action='Watch'
        ),
        TradingEntity(
            Date='2026-02-04',
            name='KKR and Singtel',
            conviction='High',
            catalyst='Acquisition of STT GDC for $5 billion indicates strong confidence in data center growth.',
            note='Integration risks post-acquisition.',
            action='Buy'
        )
    ],
    bearish=[
        TradingEntity(
            Date='2026-02-04',
            name='AMD',
            conviction='High',
            catalyst="AMD's stock plummeted 16% due to disappointing forecasts amid an AI spending boom.",
            note='Continued pressure from competition.',
            action='Sell'
        ),
        TradingEntity(
            Date='2026-02-04',
            name='Novo Nordisk',
            conviction='High',
            catalyst="Novo Nordisk's shares tumbled 18% after a surprise negative forecast for 2026.",
            note="Market share risk from Eli Lilly's growth.",
            action='Sell'
        ),
        TradingEntity(
            Date='2026-02-04',
            name='Chipotle',
            conviction='Medium',
            catalyst="Chipotle's stock has fallen 33% over the last year due to declining traffic and weak 
guidance.",
            note='Consumer spending trends may impact recovery.',
            action='Sell'
        ),
        TradingEntity(
            Date='2026-02-04',
            name='Qualcomm',
            conviction='Medium',
            catalyst="Qualcomm's stock sank as memory shortages negatively impacted its forecast.",
            note='Supply chain issues could persist.',
            action='Sell'
        ),
        TradingEntity(
            Date='2026-02-04',
            name='Washington Post',
            conviction='Medium',
            catalyst='Widespread layoffs at the Washington Post indicate financial distress amid losses.',
            note='Declining media landscape.',
            action='Avoid'
        )
    ]
)

TradingSignals(
    bullish=[
        TradingEntity(
            Date='2026-02-05',
            name='Broadcom',
            conviction='High',
            catalyst='Jefferies sees a 60% surge for Broadcom due to its edge in supplying Alphabet’s next two 
major chip generations.',
            note=None,
            action='Buy'
        ),
        TradingEntity(
            Date='2026-02-05',
            name='Ciena',
            conviction='High',
            catalyst='Ciena returns to S&P 500, predicting the fastest growth since 2011 thanks to its position in 
artificial intelligence.',
            note=None,
            action='Buy'
        ),
        TradingEntity(
            Date='2026-02-05',
            name='Gold',
            conviction='High',
            catalyst='Gold shows tentative signs of recovery after an intense sell-off, indicating potential for 
investment banks to see upward movement.',
            note=None,
            action='Buy'
        ),
        TradingEntity(
            Date='2026-02-05',
            name='Pandora',
            conviction='High',
            catalyst="Pandora's CEO addresses investor concerns about reliance on silver, leading to a positive 
market response.",
            note=None,
            action='Buy'
        )
    ],
    bearish=[
        TradingEntity(
            Date='2026-02-05',
            name='Estée Lauder',
            conviction='High',
            catalyst='Estée Lauder expects a $100 million tariff hit to full-year profitability, causing a 
significant stock decline.',
            note=None,
            action='Sell'
        ),
        TradingEntity(
            Date='2026-02-05',
            name='Qualcomm',
            conviction='High',
            catalyst='Qualcomm issues a dire warning on memory shortage impacting the entire industry, leading to a
7% stock drop.',
            note=None,
            action='Sell'
        ),
        TradingEntity(
            Date='2026-02-05',
            name='Volvo Cars',
            conviction='High',
            catalyst='Volvo Cars experiences its worst trading day ever, falling over 22% due to substantial profit
drop and weak demand.',
            note=None,
            action='Sell'
        ),
        TradingEntity(
            Date='2026-02-05',
            name='Peloton',
            conviction='High',
            catalyst='Peloton shares plunge more than 20% on weak holiday quarter and sluggish demand for new 
products.',
            note=None,
            action='Sell'
        ),
        TradingEntity(
            Date='2026-02-05',
            name='Bitcoin',
            conviction='High',
            catalyst='Bitcoin drops below $67,000 as sell-off intensifies, indicating further potential decline 
below key support levels.',
            note=None,
            action='Sell'
        )
    ]
)

[None, None, None, None, None]